In [1]:
import aliases

In [2]:
import hashlib as _hashlib
import itertools as _itertools

from everest.utilities import (
    FrozenMap as _FrozenMap,
    caching as _caching,
    word as _word,
    )


class Register:

    __slots__ = (
        '_args', '_kwargs', '_argtup', '_frozenkwargs', '_argskwargslocked',
        '_softcache', 'checker',
        )

    def __init__(self, /, checker=(lambda x: x)):
        self._argskwargslocked = False
        self._args = []
        self._kwargs = {}
        self.checker = checker

    @property
    def argskwargslocked(self):
        return self._argskwargslocked

    def register(self, *args, **kwargs):
        if self.argskwargslocked:
            raise RuntimeError("Cannot register on frozen registrar.")
        for att in ('_argtup', '_frozenkwargs'):
            if hasattr(self, att):
                delattr(self, att)
        self._args.extend(args)
        self._kwargs.update(kwargs)

    def freeze(self):
        self._argskwargslocked = True
        self._argtup = tuple(self._args)
        self._frozenkwargs = _FrozenMap(**self._kwargs)
        if (checker := self.checker) is not None:
            if not all(map(checker, self)):
                raise TypeError("Registered inputs failed check.")

    @property
    def args(self):
        try:
            return self._argtup
        except AttributeError:
            raise RuntimeError("Cannot access args until frozen.")

    @property
    def kwargs(self):
        try:
            return self._frozenkwargs
        except AttributeError:
            raise RuntimeError("Cannot access kwargs until frozen.")

    def __call__(self, *args, **kwargs):
        return self.register(*args, **kwargs)

    def _repr(self):
        return f"{type(self).__name__}({self.__str__()})"

    @_caching.softcache(None)
    def __repr__(self):
        return self._repr()

    def _str(self):
        args, kwargs = self.args, self.kwargs
        allreps = _itertools.chain(
            map(repr, args),
            map("=".join, zip(kwargs, map(repr, kwargs.values()))),
            )
        return ', '.join(allreps)

    @_caching.softcache(None)
    def __str__(self):
        return self._str()

    def _hashcode(self):
        content = str(self).encode()
        return _hashlib.md5(content).hexdigest()

    @_caching.softcache(None)
    def hashcode(self):
        return self._hashcode()

    def _hashint(self):
        return int(self.hashcode(), 16)

    @_caching.softcache(None)
    def hashint(self):
        return self._hashint()

    def _hashstr(self):
        return _word.get_random_english(
            seed=self.hashint(),
            n=2,
            )

    @_caching.softcache(None)
    def hashstr(self):
        return self._hashstr()

    def __iter__(self):
        return _itertools.chain(self.args, self.kwargs.values())

    def __len__(self):
        return len(self.args) + len(self.kwargs)

In [3]:
myreg = Register()
myreg(1, 2, c=3)
myreg.freeze()
myreg.hashstr()

'clearer-professorships'

In [21]:
from abc import ABCMeta as _ABCMeta, abstractmethod as _abstractmethod
import weakref as _weakref
import itertools as _itertools

from everest.utilities import caching as _caching


class PtolemaicMeta(_ABCMeta):
    '''
    The metaclass of all proper Ptolemaic classes.
    '''

    passleftoverparams = False

    Register = Register

    def _cls_extra_init_(cls, /):
        pass

    def __new__(meta, *args, clskwargs=None, **kwargs):
        cls = super().__new__(meta, *args, **kwargs)
        if clskwargs is None:
            cls._cls_extra_init_()
        else:
            cls._cls_extra_init_(**clskwargs)
        return cls

    def parameterise(cls, register, /, *args, **kwargs):
        raise NotImplementedError("No parameterise method has been provided.")

    def instantiate(cls, register):
        raise NotImplementedError("No instantiate method has been provided.")

    def construct(cls):
        raise NotImplementedError("No construct method has been provided.")

    @property
    def __call__(cls):
        return cls.construct


class PtolemaicBase(metaclass=PtolemaicMeta):
    '''
    The base class of all object types
    understood by the Ptolemaic system.
    '''

    @classmethod
    def _cls_extra_init_(cls, /):
        pass

    @classmethod
    def param_checker(cls, arg):
        return True

    @classmethod
    def construct(cls, *args, **kwargs):
        register = Register(checker=cls.param_checker)
        cls.parameterise(register, *args, **kwargs)
        return cls.instantiate(register)

    @classmethod
    def parameterise(cls, register, /, *args, **kwargs):
        if args or kwargs:
            if cls.passleftoverparams:
                register(*args, **kwargs)
            else:
                raise TypeError(
                    f"Leftovers from parameterisation: {args}, {kwargs}"
                    )
        register.freeze()

    @classmethod
    def param_checker(cls, arg):
        return isinstance(arg, PtolemaicBase)

    @classmethod
    def instantiate(cls, register):
        obj = object.__new__(cls)
        obj.register = register
        obj._softcache = dict()
        obj.__init__()
        return obj

    def __init__(self):
        pass

    def _repr(self):
        return f"{type(self).__name__}({self.register.__str__()})"

    @_caching.softcache(None)
    def __repr__(self):
        return self._repr()


class Primitive(metaclass=PtolemaicMeta):
    '''
    The abstract base class of all Python types
    that are acceptables as inputs
    to the Ptolemaic system.
    '''

    PRIMITIVETYPES = (
        int,
        float,
        str,
        bytes,
        bool,
        )

    @classmethod
    def __subclasshook__(cls, C):
        if cls is Primitive:
            if any(C is typ for typ in cls.PRIMITIVETYPES):
                return True
        return NotImplemented


_ = PtolemaicBase.register(Primitive)

In [27]:
class Aspect(metaclass=PtolemaicMeta):
    '''
    Aspect classes are compatible as bases for other classes.
    '''


class Singleton(Aspect):

    @classmethod
    def _cls_extra_init_(cls, /):
        cls.premade = _weakref.WeakValueDictionary()

    @classmethod
    def prekey(cls, register):
        return register.__str__()

    @classmethod
    def instantiate(cls, register):
        premade = cls.__dict__['premade']
        prekey = cls.prekey(register)
        if prekey in premade:
            return premade[prekey]
        obj = super().instantiate(register)
        premade[prekey] = obj
        return obj

In [28]:
class Ptolemaic(Singleton, PtolemaicBase):

    @classmethod
    def prekey(cls, register):
        return register.hashstr()

    def _repr(self):
        return f"{type(self).__name__}:{self.register.hashstr()}"

In [29]:
class SpriteBase(PtolemaicBase):
    '''
    The abstract base class of all types
    that are acceptable inputs to Sprites.
    '''


_ = SpriteBase.register(Primitive)


class Sprite(PtolemaicBase):
    '''
    The base class for all 'sprites',
    or classes whose inputs are restricted to being
    Primitive or Sprite types.
    '''

    @classmethod
    def _cls_extra_init_(cls, /):
        cls.premade = _weakref.WeakValueDictionary()

In [30]:
class MyPtol(Ptolemaic):

    @classmethod
    def parameterise(cls, register, a, b, c=3):
        register(a, b, c=3)
        super().parameterise(register)

In [31]:
myobj = MyPtol(1, 2)
myobj

MyPtol:clearer-professorships

In [32]:
myobj2 = MyPtol(1, 2)
myobj2

MyPtol:clearer-professorships

In [33]:
myobj2 is myobj

True

In [ ]:
#     def __hash__(self):
#         return self._hashint

#     for richop in ('eq', 'gt', 'lt', 'ge', 'le'):

#         for prefix in ('', 'r'):

#             opname = f"__{prefix}{richop}__"

#             @property
#             def richfunc(self, opname=opname):
#                 return getattr(self._hashint, opname)

#             exec(f"{opname} = richfunc")

#     del richop, prefix, opname, richfunc




# class Params(PtolemaicBase):

#     @classmethod
#     def _deduce_casename_(cls, args, kwargs):
#         0

In [ ]:

class SpriteBase(PtolemaicBase):
    '''
    The abstract base class of all types
    that are acceptable inputs to Sprites.
    '''


_ = SpriteBase.register(Primitive)


class Sprite(SpriteBase):
    '''
    The base class for all 'sprites',
    or classes whose inputs are restricted to being
    Primitive or Sprite types.
    '''